<a href="https://colab.research.google.com/github/devangi2000/Deep-Learning/blob/master/Assignment_5_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Using Pretrained ConvNets

You will follow the general machine learning workflow.

1. Examine and understand the data
2. Build an input pipeline
3. Compose our model
  * Part-1: Load in our pretrained base model (and pretrained weights)
  * Part-2: Stack our classification layers on top
4. Train our model
5. Evaluate model


In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

keras = tf.keras
tf.random.set_seed(10)

## Data preprocessing

### Data download

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

We acknowledge that the assignment question mentioned flower classification task by mistake. **You need to use cats vs dogs dataset here**. Sorry for the confusion.

In [ ]:
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Show the first two images and labels from the training set:

In [ ]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

### Format the Data



In [ ]:
IMG_SIZE = 160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

Apply this function to each item in the dataset using the map method:

In [ ]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

Now shuffle and batch the data.

In [ ]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [ ]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)


### Q6 What is the shape of one batch of data?

## Create the base model from the pre-trained convnets
You will create the base model from the **InceptionV3** model developed at Google. This is pre-trained on the ImageNet dataset, a large dataset consisting of 1.4M images and 1000 classes. ImageNet is a research training dataset with a wide variety of categories like `jackfruit` and `syringe`. This base of knowledge will help us classify cats and dogs from our specific dataset.

First, you need to pick which layer of InceptionV3 you will use for feature extraction. The very last classification layer (on "top", as most diagrams of machine learning models go from bottom to top) is not very useful.  Instead, you will follow the common practice to depend on the very last layer before the flatten operation. This layer is called the "bottleneck layer". The bottleneck layer features retain more generality as compared to the final/top layer.

First, instantiate a InceptionV3 model pre-loaded with weights trained on ImageNet. By specifying the **include_top=False** argument, you load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

In [ ]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model InceptionV3
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')


In [ ]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

### Q7 What is the shape of a new block of features converted by the feature extractor?

In [ ]:
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.summary()

## Feature extraction


### Freeze the convolutional base

In [ ]:
# Write the code here to freeze convolutional base
base_model.trainable = False
base_model.summary()

### Add a classification head

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer = keras.layers.Dense(metadata.features['label'].num_classes, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

Now stack the feature extractor, and these two layers using a `tf.keras.Sequential` model:

In [ ]:
# Write the code here to stack the all these layers

model = tf.keras.Sequential([
                                    base_model,
                                    global_average_layer,
                                    prediction_layer
])

### Q8 What is the total number trainable TF variables in the model?

### Compile the model


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Q9 What is the total numebr of Trainable parameters in this model?

### Train the model

In [ ]:
history = model.fit(train_batches,
                    epochs=5,
                    validation_data=validation_batches)

### Learning curves


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
len(model.trainable_variables)

# 2. Fine tuning


### Un-freeze the top layers of the model


In [ ]:
# Write code here for un-freezing the top layers
base_model.trainable = True

In [ ]:
# Write the code for freezing layers before layer 40 in the model
print('Number of layers in InceptionV3 : ', len(base_model.layers))

In [ ]:
fine_tune_at = 250

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

### Q10 What is the total number trainable TF variables in the model?

### Compile the model


In [ ]:
# Write the code here to stack the all these layers

model = tf.keras.Sequential([
                                    base_model,
                                    global_average_layer,
                                    prediction_layer
])

model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Q11 What is the total numebr of Trainable parameters in this model now?

In [ ]:
len(model.trainable_variables)

### Continue Train the model

In [ ]:
initial_epochs=5
total_epochs = initial_epochs + 5

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch = initial_epochs,
                         validation_data=validation_batches)

### Q12 Let the training accuracy in part-1 be x and validation accuracy in part-2 be y (both after training is done). The value of | y-x | lies between?